In [1]:
import os
import re 

import pandas as pd
from tqdm import tqdm

tqdm.pandas()

from preprocessing_utils import *

2024-03-15 22:17:16 rhodes-f6eac metapub.config[252199] WARNING NCBI_API_KEY was not set.
2024-03-15 22:17:18.261420: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-15 22:17:18.261469: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-15 22:17:18.262306: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-15 22:17:18.268729: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow 

In [2]:
DATA_PATH = '/data/01_raw_data/'

# RegulonDB Data

In [3]:
regulation_df = pd.read_csv(os.path.join(DATA_PATH,'RISet.tsv'), skiprows=41, sep='\t')
print(regulation_df.shape)
regulation_df.head(2)

(6110, 25)


,1)riId,2)riType,3)regulatorId,4)regulatorName,5)cnfName,6)tfrsID,7)tfrsLeft,8)tfrsRight,9)strand,10)tfrsSeq,...,16)tfrsDistToPm,17)firstGene,18)tfrsDistTo1Gene,19)targetTuOrGene,20)confidenceLevel,21)tfrsEvidence,22)riEvidence,23)addEvidence,24)riEvTech,25)riEvCategory
0,RDBECOLIRIC00001,tf-promoter,RDBECOLITFC00023,GadW,GadW,RDBECOLIBSC01689,3653866.0,3653885.0,forward,atcagacaggTTTACGTTACTATCAGGCATatcacctcag,...,-60.5,slp,-85.5,RDBECOLITUC03133:slp-dctR,W,COMP-AINF-SIMILAR-TO-CONSENSUS:W,EXP-IEP-GENE-EXPRESSION-ANALYSIS:W,NaN,expression|non-experimental,Classical expression|non-experimental
1,RDBECOLIRIC00002,tf-promoter,RDBECOLITFC00023,GadW,GadW,RDBECOLIBSC01690,3656835.0,3656854.0,reverse,tgaaataaaaATATCTGATTTTGATATTTTccatcaacat,...,-53.5,hdeA,-104.5,RDBECOLITUC03315:hdeAB-yhiD,S,COMP-HINF-SIMILAR-TO-CONSENSUS:W;COMP-AINF-SIM...,EXP-IEP-GENE-EXPRESSION-ANALYSIS:W,NaN,expression|non-experimental|binding in vitro,Classical expression|non-experimental|Classica...


### Prepare Dataset
#### Filter and rename columns

In [4]:
regulation_df.columns = [re.sub('^\d{1,2}\)', '', col).strip() for col in regulation_df.columns]
regulation_df = regulation_df.rename(columns={'regulatorName': 'srna_name', 
                                              'firstGene': 'target_gene',
                                              'riFunction' : 'type_of_regulation',
                                           })
regulation_df = regulation_df[['riId','regulatorId','srna_name', 'target_gene', 'targetTuOrGene', 'type_of_regulation']]
print(regulation_df.shape)
regulation_df.head(2)

(6110, 6)


,riId,regulatorId,srna_name,target_gene,targetTuOrGene,type_of_regulation
0,RDBECOLIRIC00001,RDBECOLITFC00023,GadW,slp,RDBECOLITUC03133:slp-dctR,repressor
1,RDBECOLIRIC00002,RDBECOLITFC00023,GadW,hdeA,RDBECOLITUC03315:hdeAB-yhiD,activator


In [5]:
regulator_df = pd.read_csv(os.path.join(DATA_PATH,'RegulatorSet.tsv'), skiprows=34, sep='\t')
print(regulator_df.shape)
regulator_df.head(2)

(284, 18)


,1)regulatorId,2)regulatorName,3)regulatorSynonyms,4)regulatorType,5)geneCodingForRegulator,6)regulatorActiveConformations,7)regulatorInactiveConformations,8)regulatorActiveConformationsSynonyms,9)regulatorInactiveConformationsSynonyms,10)regulatorActiveConformationsEffector,11)regulatorInactiveConformationsEffector,12)regulatorActiveConformationsEffectorSynonyms,13)regulatorInactiveConformationsEffectorSynonyms,14)symmetry,15)regulatorEvidences,16)additiveEvidences,17)confidenceLevel,18)regulatorConformationPMID
0,RDBECOLITFC00001,ExuR,"ExuR;negative regulator of exu regulon, exuT, ...",transcriptionFactor,exuR,"ExuR;ExuR-&alpha;-D-galacturonate, &alpha;-D-g...",ExuR-&alpha;-D-glucuronate,"ExuR;negative regulator of exu regulon, exuT, ...",NaN,&alpha;-D-galacturonate,&alpha;-D-glucuronate,&alpha;-D-galacturonic acid,&alpha;-D-glucuronic acid;&alpha;-D-glucopyran...,|INVERTED-REPEAT|,[EXP:W],NaN,W,6357945
1,RDBECOLITFC00002,AsnC,AsnC,transcriptionFactor,asnC,DNA-binding transcriptional dual regulator AsnC,AsnC-L-asparagine,AsnC,NaN,NaN,L-asparagine,NaN,asparagine;&alpha;-aminosuccinamic acid;(-)-as...,|INVERTED-REPEAT|,[COMP-HINF-FN-FROM-SEQ:W][EXP-IEP:W][EXP-IPI:W],NaN,W,6357950; 7686882; 16528101; 2864330


In [6]:
regulator_df.columns = [re.sub('^\d{1,2}\)', '', col).strip() for col in regulator_df.columns]
regulator_df = regulator_df.rename(columns={'regulatorName': 'srna_name', 
                                            'regulatorSynonyms':'srna_synonyms', 
                                            'geneCodingForRegulator':'srna_code', 
                                            'regulatorConformationPMID':'PMIDs'
                                           })

regulator_df = regulator_df[['regulatorId','srna_name', 'srna_synonyms', 'srna_code', 'PMIDs']]
regulator_df.head(2)

,regulatorId,srna_name,srna_synonyms,srna_code,PMIDs
0,RDBECOLITFC00001,ExuR,"ExuR;negative regulator of exu regulon, exuT, ...",exuR,6357945
1,RDBECOLITFC00002,AsnC,AsnC,asnC,6357950; 7686882; 16528101; 2864330


In [7]:
regulation_df = regulation_df.merge(regulator_df, how='left')
print(regulation_df.shape)
regulation_df.head(3)

(6110, 9)


,riId,regulatorId,srna_name,target_gene,targetTuOrGene,type_of_regulation,srna_synonyms,srna_code,PMIDs
0,RDBECOLIRIC00001,RDBECOLITFC00023,GadW,slp,RDBECOLITUC03133:slp-dctR,repressor,YhiW;GadW,gadW,12446650
1,RDBECOLIRIC00002,RDBECOLITFC00023,GadW,hdeA,RDBECOLITUC03315:hdeAB-yhiD,activator,YhiW;GadW,gadW,12446650
2,RDBECOLIRIC00003,RDBECOLITFC00088,GalR,galP,RDBECOLITUC02515:galP,repressor,GalR,galR,12169637; 15533432; 15807530; 17630975; 921880...


In [8]:
regulation_df = regulation_df[(~regulation_df.srna_name.isnull())&(~regulation_df.target_gene.isnull())&(~regulation_df.type_of_regulation.isnull())]
regulation_df.drop_duplicates()
print(regulation_df.shape)
regulation_df.head(3)

(6088, 9)


,riId,regulatorId,srna_name,target_gene,targetTuOrGene,type_of_regulation,srna_synonyms,srna_code,PMIDs
0,RDBECOLIRIC00001,RDBECOLITFC00023,GadW,slp,RDBECOLITUC03133:slp-dctR,repressor,YhiW;GadW,gadW,12446650
1,RDBECOLIRIC00002,RDBECOLITFC00023,GadW,hdeA,RDBECOLITUC03315:hdeAB-yhiD,activator,YhiW;GadW,gadW,12446650
2,RDBECOLIRIC00003,RDBECOLITFC00088,GalR,galP,RDBECOLITUC02515:galP,repressor,GalR,galR,12169637; 15533432; 15807530; 17630975; 921880...


In [9]:
regulation_df = regulation_df[(~regulation_df.PMIDs.isnull())&(regulation_df.PMIDs!=' ')]
regulation_df.drop_duplicates()
print(regulation_df.shape)
regulation_df.head(3)

(3749, 9)


,riId,regulatorId,srna_name,target_gene,targetTuOrGene,type_of_regulation,srna_synonyms,srna_code,PMIDs
0,RDBECOLIRIC00001,RDBECOLITFC00023,GadW,slp,RDBECOLITUC03133:slp-dctR,repressor,YhiW;GadW,gadW,12446650
1,RDBECOLIRIC00002,RDBECOLITFC00023,GadW,hdeA,RDBECOLITUC03315:hdeAB-yhiD,activator,YhiW;GadW,gadW,12446650
2,RDBECOLIRIC00003,RDBECOLITFC00088,GalR,galP,RDBECOLITUC02515:galP,repressor,GalR,galR,12169637; 15533432; 15807530; 17630975; 921880...


In [10]:
pmids = regulation_df.PMIDs.to_list()
pmids = list(set([pmid.strip() for entry in pmids for pmid in entry.strip().split(';')]))
print(f'Found {len(pmids)} unique PMIDs')

Found 510 unique PMIDs


### Get Fulltexts from PubMed

In [11]:
import metapub

In [12]:
fulltext_df = pd.DataFrame(columns=["PMID", "fulltext"])
fulltext_df['PMID'] = pmids
fulltext_df['fulltext'] = fulltext_df.PMID.progress_apply(lambda x: get_fulltext(x))
fulltext_df.head(2)

  0%|▏                                          | 2/510 [00:00<02:21,  3.59it/s]

Error fetching full text: expected string or bytes-like object


  2%|▉                                         | 11/510 [00:15<10:41,  1.28s/it]

Error fetching full text: expected string or bytes-like object


  2%|▉                                         | 12/510 [00:16<09:40,  1.17s/it]

...

100%|█████████████████████████████████████████| 510/510 [16:12<00:00,  1.13s/it]

Error fetching full text: expected string or bytes-like object


100%|█████████████████████████████████████████| 510/510 [16:13<00:00,  1.91s/it]

Error fetching full text: expected string or bytes-like object


,PMID,fulltext
0,8098993,
1,12446641,"JOURNAL OF BACTERIOLOGY, Dec. 2002, p. 6906–69..."


In [13]:
fulltext_df = fulltext_df[fulltext_df.fulltext!=""]
print(fulltext_df.shape)
fulltext_df.head(3)

(262, 2)


,PMID,fulltext
1,12446641,"JOURNAL OF BACTERIOLOGY, Dec. 2002, p. 6906–69..."
2,32753505,RESEARCH ARTICLE\nMolecular Biology and Physio...
3,30137486,"10682–10696 Nucleic Acids Research, 2018, Vol...."


In [14]:
fulltext_df['fulltext'] = fulltext_df.fulltext.apply(lambda x: clean_fulltexts(x))

AttributeError: 'str' object has no attribute 'decode'

In [ ]:
fulltext_df = pd.read_parquet('./fulltexts_regulondb.pq')

In [ ]:
regulation_df['PMIDs'] = regulation_df['PMIDs'].str.split(';')
regulation_df = regulation_df.explode('PMIDs').reset_index(drop=True)
regulation_df['PMIDs'] = regulation_df.PMIDs.str.strip()                                                          
print(regulation_df.shape)
regulation_df.head(3)

In [ ]:
fulltext_df = fulltext_df.merge(regulation_df, left_on='PMID', right_on='PMIDs', how='left')
fulltext_df.drop_duplicates()
print(fulltext_df.shape)
fulltext_df.head(3)

In [ ]:
fulltext_df.type_of_regulation.unique()

In [ ]:
len(fulltext_df.srna_name.unique())

In [ ]:
len(fulltext_df.target_gene.unique())

In [ ]:
fulltext_df.to_parquet('./fulltexts_regulondb.pq')

In [ ]:
fulltext_df = pd.read_parquet('./fulltexts_regulondb.pq')
fulltext_df.head(3)

# sRNA TarBase Data

In [ ]:
srna_targetbase_df = pd.read_csv('./sRNATarBase3_w_target_from_1_to_terminal.csv')
print(srna_targetbase_df.shape)
srna_targetbase_df.head(3)

In [ ]:
srna_targetbase_df.columns

In [ ]:
srna_targetbase_df = srna_targetbase_df[['ID', 
                                         'SRNA_NAME', 
                                         'SRNA_ALIAS', 
                                         'TARGET_NAME', 
                                         'TARGET_ALIAS', 
                                         'STRAIN_NAME',
                                         'REGULATION', 
                                         'PMID']]
srna_targetbase_df = srna_targetbase_df.rename(columns={'SRNA_NAME':'srna_name', 
                                                        'SRNA_ALIAS':'srna_synonyms', 
                                                        'TARGET_NAME':'target_gene', 
                                                        'TARGET_ALIAS':'target_gene_synonyms', 
                                                        'STRAIN_NAME':'strain_name', 
                                                        'REGULATION':'regulation'})
print(srna_targetbase_df.shape)
srna_targetbase_df.head(3)

In [ ]:
tmp_srna_targetbase_df = srna_targetbase_df[srna_targetbase_df.PMID.str.contains(',')]
srna_targetbase_df = srna_targetbase_df[~srna_targetbase_df.PMID.str.contains(',')]

tmp_srna_targetbase_df['PMID'] = tmp_srna_targetbase_df['PMID'].str.split(';')
tmp_srna_targetbase_df = tmp_srna_targetbase_df.explode('PMID').reset_index(drop=True)

srna_targetbase_df = pd.concat([srna_targetbase_df, tmp_srna_targetbase_df])

srna_targetbase_df['PMID'] = srna_targetbase_df.PMID.str.strip()   
print(srna_targetbase_df.shape)
srna_targetbase_df.head(3)

In [ ]:
srna_targetbase_df['fulltext'] = srna_targetbase_df.PMID.progress_apply(lambda x: get_fulltext(x))

In [ ]:
srna_targetbase_df['fulltext'] = srna_targetbase_df.fulltext.apply(lambda x: clean_fulltexts(x) if x!="" else "")

In [ ]:
srna_targetbase_df[srna_targetbase_df.fulltext!=''].shape

In [ ]:
srna_targetbase_df.regulation.unique()

In [ ]:
srna_targetbase_df[srna_targetbase_df.regulation!="None"]

In [ ]:
srna_targetbase_df.to_parquet('./fulltexts_srna_targetbase.pq')